In [ ]:
sample_size <- 20000
n_throws <- 10
m_heads <- 3

# Choose a random coin (from the mixture of betas distribution)
source_beta <- sample(c(1,2,3), replace = TRUE, size = sample_size, prob=c(0.5, 0.2, 0.3))

# Generate thetas values choosing coins from the corresponding beta distribution
thetas <- c(rbeta(sample_size,10,20)[source_beta == 1], 
            rbeta(sample_size,15,15)[source_beta == 2], 
            rbeta(sample_size,20,10)[source_beta == 3])

# Simulated prior
hist(thetas, breaks=50)

# Number of heads in m trials
heads <- rbinom(n = sample_size, size = n_throws, prob = thetas)

# Absolute frequencies
print("Absolute frequency of m heads:")
table(heads)
# Relative frequencies. 
# The relative frequency of 3 heads is the X-marginal for X=m=3.
# Obtaining m=3 heads is not the most likely outcome, based on this prior
print("Relative frequency of m heads:")
f <- table(heads)/sum(table(heads))
f
matplot(x=names(f),y=f,type="h",lwd=7, lty=1,
        xlab="x",ylab="Rel. Freqs..",
        main="Simulated prior predictive pmf")

# Cases in which we get m heads
m.heads.idx <- heads == m_heads

# Proportion of cases in which we get m heads
print("Proportion of cases that we keep to calculate the posterior:")
sum(m.heads.idx)/length(m.heads.idx)

# Thetas that have generated m heads
thetas.m.heads <- thetas[m.heads.idx]

# Based on this simulation, the MAP is around 0.3
hist(thetas.m.heads, breaks=50, freq=FALSE, main = "Histogram of thetas producing m heads")



In [ ]:
# Finding the maximum a posteriori (MAP)
thetas.m.heads.density <-density(thetas.m.heads)
Sim.MAP <- thetas.m.heads.density$x[which.max(thetas.m.heads.density$y)]
hist(thetas.m.heads, breaks=50, freq=FALSE, main="Histogram of thetas producing m heads")
abline(v=Sim.MAP,col="DarkRed",lwd=4)
text(0.42, 0.5, "Simulated MAP", col="DarkRed")
print(paste0("Simulated MAP: ", round(Sim.MAP,3)))

In [ ]:
# Simulated Expectation
print(paste0("Simulation based posterior expectation:",round(mean(thetas.m.heads),3)))

# Simulated Variance
print(paste0("Simulation based posterior variance:",round(var(thetas.m.heads),3)))


# Simulated Quantiles
print("Simulation based posterior quantiles:")
quantiles <- quantile(thetas.m.heads,c(0,0.25,0.50,0.75,1))
round(quantiles,4)

In [ ]:
# 90% credible interval for theta
y.max<-max(thetas.m.heads.density$y)
gamma <- 0.9
Sim.gamma <-function(s){
    CI.thetas<-(thetas.m.heads.density$x)[which(thetas.m.heads.density$y>=(y.max-s))]
    return(sum((min(CI.thetas)<=thetas.m.heads)&(thetas.m.heads<=max(CI.thetas)))/length(thetas.m.heads)-gamma)
    }
G.s <- uniroot(Sim.gamma,c(0,y.max))
Sim.s<-G.s$root
round(Sim.s,4)

CI.thetas<-(thetas.m.heads.density$x)[which(thetas.m.heads.density$y>=Sim.s)]
Credible.Interval <-c(min(CI.thetas),max(CI.thetas))
cat(sprintf("Credible interval 03 (%5.2f %%): (%6.4f, %6.4f)",gamma*100, 
            round(Credible.Interval[1],4),round(Credible.Interval[2],4)))
width <-diff(Credible.Interval)
round(width,4)